In [5]:
# # [데이터 분할을 위한 폴더 생성]

# import os
# import shutil

# original_dataset_dir = '/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec88'
# classes_list = os.listdir(original_dataset_dir)

# base_dir = '/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted'
# os.mkdir(base_dir)

# train_dir = os.path.join(base_dir, 'train')
# os.mkdir(train_dir)
# validation_dir = os.path.join(base_dir, 'val')
# os.mkdir(validation_dir)
# test_dir = os.path.join(base_dir, 'test')
# os.mkdir(test_dir)

# # train,validation,test 폴더 하위에 각각 클래스 목록 폴더 생성
# for clss in classes_list:
#     os.mkdir(os.path.join(train_dir,clss))
#     os.mkdir(os.path.join(validation_dir,clss))
#     os.mkdir(os.path.join(test_dir,clss))

In [6]:
# # [데이터 분할과 클래스별 데이터 수 확인]

# import math

# for clss in classes_list:
#     path = os.path.join(original_dataset_dir,clss)
#     fnames = os.listdir(path)
    
#     train_size = math.floor(len(fnames)*0.6)
#     validation_size = math.floor(len(fnames)*0.2)
#     test_size = math.floor(len(fnames)*0.2)
    
#     train_fnames = fnames[:train_size]
#     print('Train size(',clss,'): ', len(train_fnames))
#     for fname in train_fnames:
#         src = os.path.join(path, fname)
#         dst = os.path.join(os.path.join(train_dir, clss), fname)
#         shutil.copyfile(src,dst)
        
#     validation_fnames = fnames[train_size:(validation_size + train_size)]
#     print('Validation size(',clss,'): ', len(validation_fnames))
#     for fname in validation_fnames:
#         src = os.path.join(path, fname)
#         dst = os.path.join(os.path.join(validation_dir,clss),fname)
#         shutil.copyfile(src,dst)
        
#     test_fnames = fnames[(train_size+validation_size):(validation_size+train_size+test_size)]
#     print('Test size(',clss,'): ', len(test_fnames))
#     for fname in test_fnames:
#         src = os.path.join(path, fname)
#         dst = os.path.join(os.path.join(test_dir, clss),fname)
#         shutil.copyfile(src,dst)

In [7]:
# [베이스라인 모델 학습을 위한 준비]

import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE = 256
EPOCH = 10

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

train_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted/train', transform = transform_base)
test_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted/test', transform = transform_base)


# Dataloader는 이미지 데이터를 미니 배치 단위로 분리하는 역할
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
val_loader = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)

In [8]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os

data_transforms = {
    'train':transforms.Compose([
        transforms.Resize([64,64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ]),
    'test':transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ])
}

data_dir = '/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted'

image_datasets = {x: ImageFolder(root=os.path.join(data_dir,x),
                                 transform=data_transforms[x]) for x in ['train','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = BATCH_SIZE, shuffle=True, num_workers = 4) for x in ['train','test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

In [9]:
# [Pre-Trained Model 불러오기]

from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

resnet = models.resnet50(pretrained=True)
# 이 프로젝트에선 33개로 분류해야함. 마지막 Layer 출력 채널 수는 33개여야함.
# 하지만 ResNet의 마지막 Layer 출력 채널 수가 33개가 아님. 이 프로젝트의 주제에 맞추고자 모델의 마지막 Fully Connected Layer 대신 출력 채널의 수가 33개인 새로운 Layer를 추가할 예정
# in_features : 해당 Layer의 입력 채널 수
num_ftrs = resnet.fc.in_features
# 불러온 모델의 마지막 Fully Connected Layer를 새로운 Layer로 교체. 입력 채널의 수는 기존 Layer와 동일하고, 출력 채널 수는 33개로 수정
resnet.fc = nn.Linear(num_ftrs, 88)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Transfer Learning에선 일부 Layer의 Parameter만을 업데이트 해야하기 때문에 requires_grad = True로 설정된 Layer의 Parameter에만 적용
optimizer_ft = optim.Adam(filter(lambda p:p.requires_grad, resnet.parameters()), lr = 0.001)

# StepLR() 메서드는 Epoch에 따라 Learning Rate를 변경하는 역할
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [10]:
# [Pre-Trained Model 일부 Layer Freeze 하기]

ct = 0 # 해당 Layer가 몇 번째인지
for child in resnet.children(): # resnet.children()은 생성한 resnet모델의 모든 Layer 정보를 담고 있음
    ct += 1 # 다음 Layer를 지칭하도록 ct변수를 1씩 증가
    if ct < 6: # 1~5번 Layer는 업데이트 되지 않도록 고정. 6~10번 Layer는 업데이트
        for param in child.parameters():
            param.requires_grad = False

In [11]:
# [Transfer Learning 모델 학습과 검증을 위한 함수]

import time
import copy

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())  
    best_acc = 0.0  
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch+1)) 
        since = time.time()                                     
        for phase in ['train', 'test']: 
            if phase == 'train': 
                model.train()  # 학습 모드
            else:
                model.eval() # 검증 모드
 
            running_loss = 0.0  
            running_corrects = 0  
 
            
            for inputs, labels in dataloaders[phase]: 
                inputs = inputs.to(DEVICE)  
                labels = labels.to(DEVICE)  
                
                optimizer.zero_grad() 
                
                with torch.set_grad_enabled(phase == 'train'): # 학습 단계에서만 모델 Gradient 업데이트, 검증 단계에서는 X
                    outputs = model(inputs)  
                    _, preds = torch.max(outputs, 1) 
                    loss = criterion(outputs, labels)  
    
                    if phase == 'train':   
                        loss.backward()
                        optimizer.step()
 
                running_loss += loss.item() * inputs.size(0)  
                running_corrects += torch.sum(preds == labels.data)  
            if phase == 'train':  
                scheduler.step()
 
            epoch_loss = running_loss/dataset_sizes[phase]  
            epoch_acc = running_corrects.double()/dataset_sizes[phase]  
 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)) 
 
          
            if phase == 'test' and epoch_acc > best_acc: 
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
 
        time_elapsed = time.time() - since  
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))
 
    model.load_state_dict(best_model_wts) 

    return model

In [12]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH) 

torch.save(model_resnet50, 'resnet50.pt')

-------------- epoch 1 ----------------
train Loss: 1.5084 Acc: 0.4664
test Loss: 1.2845 Acc: 0.6210
Completed in 1m 8s
-------------- epoch 2 ----------------
train Loss: 0.8222 Acc: 0.6886
test Loss: 1.8591 Acc: 0.5236
Completed in 1m 8s
-------------- epoch 3 ----------------
train Loss: 0.5403 Acc: 0.8022
test Loss: 1.7533 Acc: 0.5227
Completed in 1m 8s
-------------- epoch 4 ----------------
train Loss: 0.4012 Acc: 0.8600
test Loss: 1.3821 Acc: 0.6085
Completed in 1m 8s
-------------- epoch 5 ----------------
train Loss: 0.2868 Acc: 0.8973
test Loss: 2.1057 Acc: 0.5545
Completed in 1m 8s
-------------- epoch 6 ----------------
train Loss: 0.2104 Acc: 0.9276
test Loss: 1.4635 Acc: 0.6393
Completed in 1m 8s
-------------- epoch 7 ----------------
train Loss: 0.1995 Acc: 0.9326
test Loss: 1.8102 Acc: 0.6027
Completed in 1m 8s
-------------- epoch 8 ----------------
train Loss: 0.0903 Acc: 0.9716
test Loss: 1.7663 Acc: 0.5709
Completed in 1m 8s
-------------- epoch 9 ----------------


In [13]:
# # [Transfer Learning모델 평가를 위한 전처리]

# transform_resNet = transforms.Compose([
#         transforms.Resize([224,224]),    
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
#     ])
    
# test_resNet = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted/test', transform=transform_resNet) 
# test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [14]:
# # [모델 평가를 위한 함수]

# def evaluate(model, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
    
#     with torch.no_grad(): # torch.no_grad() : Parameter 업데이트 중단
#         for data, target in test_loader:
#             data, target = data.to(DEVICE), target.to(DEVICE)
#             output = model(data)
            
#             test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            
#             pred = output.max(1, keepdim=True)[1] # Test 데이터가 33개의 클래스에 속할 각각의 확률값이 output으로 출력. 가장 높은 가진 인덱스를 예측값으로 저장
#             correct += pred.eq(target.view_as(pred)).sum().item() # pred와 target.view_as(pred)가 일치하면 1, 일치하지 않으면 0 반환
            
#     test_loss /= len(test_loader.dataset)
#     test_accuracy = 100. * correct / len(test_loader.dataset)
#     return test_loss, test_accuracy

In [15]:
# # [Transfer Learning 모델 성능 평가하기]

# resnet50=torch.load('/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/resnet50.pt') 
# resnet50.eval()  
# test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

# print('ResNet test acc:  ', test_accuracy)